In [1]:
#!pip3 install ktrain

In [2]:
import ktrain
from ktrain import text

ktrain.__version__

'0.12.3'

In [3]:
import tensorflow as tf

dataset = tf.keras.utils.get_file(
    fname="aclImdb.tar.gz", 
    origin="http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz", 
    extract=True,
)

84131840/84125825 [==============================] - 13s 0us/step


In [4]:
import os.path
IMDB_DATADIR = os.path.join(os.path.dirname(dataset), 'aclImdb')
print(IMDB_DATADIR)

/Users/seanmhiggins/.keras/datasets/aclImdb


In [15]:
MODEL_NAME = 'distilbert'
trn, val, preproc = text.texts_from_folder(IMDB_DATADIR, 
                                          maxlen=500, 
                                          preprocess_mode=MODEL_NAME,
                                          train_test_names=['train', 
                                                            'test'],
                                          classes=['pos','neg'])

detected encoding: utf-8
preprocessing train...
language: en
train sequence lengths:
	mean : 234
	95percentile : 598
	99percentile : 913


preprocessing test...
language: en
test sequence lengths:
	mean : 229
	95percentile : 582
	99percentile : 901


In [16]:
text.print_text_classifiers()

fasttext: a fastText-like model [http://arxiv.org/pdf/1607.01759.pdf]
logreg: logistic regression using a trainable Embedding layer
nbsvm: NBSVM model [http://www.aclweb.org/anthology/P12-2018]
bigru: Bidirectional GRU with pretrained word vectors [https://arxiv.org/abs/1712.09405]
standard_gru: simple 2-layer GRU with randomly initialized embeddings
bert: Bidirectional Encoder Representations from Transformers (BERT) [https://arxiv.org/abs/1810.04805]
distilbert: distilled, smaller, and faster BERT from Hugging Face [https://arxiv.org/abs/1910.01108]


In [36]:
model = text.text_classifier(MODEL_NAME, train_data=trn, preproc=preproc)
learner = ktrain.get_learner(model,train_data=trn, val_data=val, batch_size=6)

Is Multi-Label? False
maxlen is 500



done.


In [37]:
learner.fit_onecycle(2e-5, 10)



begin training using onecycle policy with max lr of 2e-05...
Train for 4167 steps, validate for 782 steps
4167/4167 [==============================] - 1193s 286ms/step - loss: 0.2750 - accuracy: 0.8832 - val_loss: 0.1820 - val_accuracy: 0.9300


In [ ]:
# step 7 make predictions on new data
predictor = ktrain.get_predictor(learner.model, preproc=t)
predictor.predict('Jesus Christ is the central figure of Christianity.')

In [ ]:
y_pred_raw = predictor.predict(X_test)

In [ ]:
classes = predictor.get_classes()

classes.index(y_pred_raw[0])

In [ ]:
classes = predictor.get_classes()

y_pred = []
for i in range(len(y_pred_raw)):
  sample = y_pred_raw[i]
  y_pred.append(classes.index(sample))

In [ ]:
from sklearn.metrics import accuracy_score
acc = accuracy_score(y_test, y_pred)

acc

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

cm

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))